In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit

In [2]:
X, y, site = load_data()

2022-07-07 02:24:05.143732: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sklearn.utils import shuffle, resample

In [4]:
from sklearn.metrics import r2_score, median_absolute_error

In [5]:
model_dict = {
#   "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  "mlp4": {"model": mlp4, "lr": 0.001},
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [6]:
n_runs = 10

In [7]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1502, 1334, 1576, 469, 1795])

In [8]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None, train_size=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # If train_size is set, select train_size subjects to be the training data:
    if train_size is not None:
        X_train, y_train, site_train = resample(X_train, y_train, site_train, replace=False, n_samples=train_size, random_state=random_state)
    
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    
    return pd.DataFrame(result), pd.DataFrame(dict(y_pred=y_pred.squeeze(), y_test=y_test))

In [9]:
dfs_eval = []
dfs_pred = []

In [10]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=175)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("mlp4_5_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("mlp4_5_pred.csv")

##################################################
model:  mlp4
run:  0
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 flatten (Flatten)           (None, 7200)              0         
                                                                 
 dropout (Dropout)           (None, 7200)              0         
                                                                 
 dense (Dense)               (None, 500)               3600500   
                                                                 
 dropout_1 (Dropout)         (None, 500)               0         
                                                                 
 dense_1 (Dense)             (None, 500)               250500    
                                                       

2/2 [==============================] - 0s 47ms/step - loss: 11.8557 - mean_squared_error: 11.8557 - rmse: 3.4432 - mean_absolute_error: 2.6106 - val_loss: 14.2654 - val_mean_squared_error: 14.2654 - val_rmse: 3.7770 - val_mean_absolute_error: 2.6288 - lr: 0.0010
Epoch 25/1000
2/2 [==============================] - 0s 96ms/step - loss: 11.1571 - mean_squared_error: 11.1571 - rmse: 3.3402 - mean_absolute_error: 2.5749 - val_loss: 13.6092 - val_mean_squared_error: 13.6092 - val_rmse: 3.6891 - val_mean_absolute_error: 2.5712 - lr: 0.0010
Epoch 26/1000
2/2 [==============================] - 0s 34ms/step - loss: 12.0255 - mean_squared_error: 12.0255 - rmse: 3.4678 - mean_absolute_error: 2.7732 - val_loss: 13.3701 - val_mean_squared_error: 13.3701 - val_rmse: 3.6565 - val_mean_absolute_error: 2.5493 - lr: 0.0010
Epoch 27/1000
2/2 [==============================] - 0s 62ms/step - loss: 11.3312 - mean_squared_error: 11.3312 - rmse: 3.3662 - mean_absolute_error: 2.5991 - val_loss: 12.2823 - val_

Epoch 54/1000
2/2 [==============================] - 0s 49ms/step - loss: 11.6243 - mean_squared_error: 11.6243 - rmse: 3.4094 - mean_absolute_error: 2.6165 - val_loss: 15.5639 - val_mean_squared_error: 15.5639 - val_rmse: 3.9451 - val_mean_absolute_error: 2.7616 - lr: 0.0010
Epoch 55/1000
2/2 [==============================] - 0s 79ms/step - loss: 12.3018 - mean_squared_error: 12.3018 - rmse: 3.5074 - mean_absolute_error: 2.7463 - val_loss: 12.7759 - val_mean_squared_error: 12.7759 - val_rmse: 3.5743 - val_mean_absolute_error: 2.4924 - lr: 0.0010
Epoch 56/1000
2/2 [==============================] - 0s 55ms/step - loss: 10.1179 - mean_squared_error: 10.1179 - rmse: 3.1809 - mean_absolute_error: 2.4032 - val_loss: 13.2626 - val_mean_squared_error: 13.2626 - val_rmse: 3.6418 - val_mean_absolute_error: 2.5369 - lr: 0.0010
Epoch 57/1000
2/2 [==============================] - 0s 49ms/step - loss: 9.9721 - mean_squared_error: 9.9721 - rmse: 3.1579 - mean_absolute_error: 2.4766 - val_loss: 14

Epoch 83/1000
2/2 [==============================] - 0s 49ms/step - loss: 11.2642 - mean_squared_error: 11.2642 - rmse: 3.3562 - mean_absolute_error: 2.6517 - val_loss: 11.7668 - val_mean_squared_error: 11.7668 - val_rmse: 3.4303 - val_mean_absolute_error: 2.4425 - lr: 5.0000e-04
Epoch 84/1000
2/2 [==============================] - 0s 50ms/step - loss: 9.3583 - mean_squared_error: 9.3583 - rmse: 3.0591 - mean_absolute_error: 2.4507 - val_loss: 12.2834 - val_mean_squared_error: 12.2834 - val_rmse: 3.5048 - val_mean_absolute_error: 2.4553 - lr: 5.0000e-04
Epoch 85/1000
2/2 [==============================] - 0s 45ms/step - loss: 10.4930 - mean_squared_error: 10.4930 - rmse: 3.2393 - mean_absolute_error: 2.5427 - val_loss: 14.1481 - val_mean_squared_error: 14.1481 - val_rmse: 3.7614 - val_mean_absolute_error: 2.6052 - lr: 5.0000e-04
Epoch 86/1000
2/2 [==============================] - 0s 54ms/step - loss: 10.9765 - mean_squared_error: 10.9765 - rmse: 3.3131 - mean_absolute_error: 2.4774 - 

1/2 [==============>...............] - ETA: 0s - loss: 10.6024 - mean_squared_error: 10.6024 - rmse: 3.2561 - mean_absolute_error: 2.4236
Epoch 00111: ReduceLROnPlateau reducing learning rate to 0.0001250000059371814.
2/2 [==============================] - 0s 81ms/step - loss: 10.3444 - mean_squared_error: 10.3444 - rmse: 3.2163 - mean_absolute_error: 2.4067 - val_loss: 11.8836 - val_mean_squared_error: 11.8836 - val_rmse: 3.4473 - val_mean_absolute_error: 2.4212 - lr: 2.5000e-04
Epoch 112/1000
2/2 [==============================] - 0s 98ms/step - loss: 9.2829 - mean_squared_error: 9.2829 - rmse: 3.0468 - mean_absolute_error: 2.3692 - val_loss: 11.2702 - val_mean_squared_error: 11.2702 - val_rmse: 3.3571 - val_mean_absolute_error: 2.3941 - lr: 1.2500e-04
Epoch 113/1000
2/2 [==============================] - 0s 33ms/step - loss: 8.7008 - mean_squared_error: 8.7008 - rmse: 2.9497 - mean_absolute_error: 2.2846 - val_loss: 10.8465 - val_mean_squared_error: 10.8465 - val_rmse: 3.2934 - val_

...

Epoch 186/1000
2/2 [==============================] - 0s 36ms/step - loss: 8.1940 - mean_squared_error: 8.1940 - rmse: 2.8625 - mean_absolute_error: 2.2212 - val_loss: 6.7426 - val_mean_squared_error: 6.7426 - val_rmse: 2.5967 - val_mean_absolute_error: 2.1142 - lr: 6.2500e-05
Epoch 187/1000
2/2 [==============================] - 0s 37ms/step - loss: 10.0465 - mean_squared_error: 10.0465 - rmse: 3.1696 - mean_absolute_error: 2.4649 - val_loss: 6.8608 - val_mean_squared_error: 6.8608 - val_rmse: 2.6193 - val_mean_absolute_error: 2.1100 - lr: 6.2500e-05
Epoch 188/1000
2/2 [==============================] - 0s 65ms/step - loss: 8.5617 - mean_squared_error: 8.5617 - rmse: 2.9260 - mean_absolute_error: 2.3661 - val_loss: 7.4552 - val_mean_squared_error: 7.4552 - val_rmse: 2.7304 - val_mean_absolute_error: 2.1602 - lr: 6.2500e-05
Epoch 189/1000
2/2 [==============================] - 0s 36ms/step - loss: 8.3214 - mean_squared_error: 8.3214 - rmse: 2.8847 - mean_absolute_error: 2.2466 - val_lo

2/2 [==============================] - 2s 408ms/step - loss: 149.2496 - mean_squared_error: 149.2496 - rmse: 12.2168 - mean_absolute_error: 11.4256 - val_loss: 15.9973 - val_mean_squared_error: 15.9973 - val_rmse: 3.9997 - val_mean_absolute_error: 3.3557 - lr: 0.0010
Epoch 2/1000
2/2 [==============================] - 0s 128ms/step - loss: 19.7046 - mean_squared_error: 19.7046 - rmse: 4.4390 - mean_absolute_error: 3.4342 - val_loss: 8.4862 - val_mean_squared_error: 8.4862 - val_rmse: 2.9131 - val_mean_absolute_error: 2.3487 - lr: 0.0010
Epoch 3/1000
2/2 [==============================] - 0s 33ms/step - loss: 17.8374 - mean_squared_error: 17.8374 - rmse: 4.2234 - mean_absolute_error: 3.3739 - val_loss: 8.6966 - val_mean_squared_error: 8.6966 - val_rmse: 2.9490 - val_mean_absolute_error: 2.2779 - lr: 0.0010
Epoch 4/1000
2/2 [==============================] - 0s 66ms/step - loss: 21.0298 - mean_squared_error: 21.0298 - rmse: 4.5858 - mean_absolute_error: 3.8075 - val_loss: 24.1276 - val_m

Epoch 31/1000
2/2 [==============================] - 0s 51ms/step - loss: 15.3740 - mean_squared_error: 15.3740 - rmse: 3.9210 - mean_absolute_error: 3.2332 - val_loss: 9.9150 - val_mean_squared_error: 9.9150 - val_rmse: 3.1488 - val_mean_absolute_error: 2.5943 - lr: 0.0010
Epoch 32/1000
2/2 [==============================] - 0s 39ms/step - loss: 15.3361 - mean_squared_error: 15.3361 - rmse: 3.9161 - mean_absolute_error: 3.1928 - val_loss: 12.0794 - val_mean_squared_error: 12.0794 - val_rmse: 3.4755 - val_mean_absolute_error: 2.9089 - lr: 0.0010
Epoch 33/1000
1/2 [==============>...............] - ETA: 0s - loss: 15.4781 - mean_squared_error: 15.4781 - rmse: 3.9342 - mean_absolute_error: 3.1340
Epoch 00033: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
2/2 [==============================] - 0s 35ms/step - loss: 15.0430 - mean_squared_error: 15.0430 - rmse: 3.8785 - mean_absolute_error: 3.0827 - val_loss: 9.5109 - val_mean_squared_error: 9.5109 - val_rmse: 3.0840 - 

Epoch 59/1000
2/2 [==============================] - 0s 47ms/step - loss: 14.0130 - mean_squared_error: 14.0130 - rmse: 3.7434 - mean_absolute_error: 3.1329 - val_loss: 10.1182 - val_mean_squared_error: 10.1182 - val_rmse: 3.1809 - val_mean_absolute_error: 2.6230 - lr: 2.5000e-04
Epoch 60/1000
2/2 [==============================] - 0s 51ms/step - loss: 15.1553 - mean_squared_error: 15.1553 - rmse: 3.8930 - mean_absolute_error: 3.2583 - val_loss: 10.3892 - val_mean_squared_error: 10.3892 - val_rmse: 3.2232 - val_mean_absolute_error: 2.6559 - lr: 2.5000e-04
Epoch 61/1000
2/2 [==============================] - 0s 27ms/step - loss: 13.1680 - mean_squared_error: 13.1680 - rmse: 3.6288 - mean_absolute_error: 2.9677 - val_loss: 10.5643 - val_mean_squared_error: 10.5643 - val_rmse: 3.2503 - val_mean_absolute_error: 2.6817 - lr: 2.5000e-04
Epoch 62/1000
2/2 [==============================] - 0s 34ms/step - loss: 13.2917 - mean_squared_error: 13.2917 - rmse: 3.6458 - mean_absolute_error: 2.9970 

2/2 [==============================] - 0s 47ms/step - loss: 14.1838 - mean_squared_error: 14.1838 - rmse: 3.7661 - mean_absolute_error: 3.0340 - val_loss: 10.3236 - val_mean_squared_error: 10.3236 - val_rmse: 3.2130 - val_mean_absolute_error: 2.6492 - lr: 1.2500e-04
Epoch 88/1000
2/2 [==============================] - 0s 28ms/step - loss: 13.4059 - mean_squared_error: 13.4059 - rmse: 3.6614 - mean_absolute_error: 3.0349 - val_loss: 10.1662 - val_mean_squared_error: 10.1662 - val_rmse: 3.1884 - val_mean_absolute_error: 2.6293 - lr: 1.2500e-04
Epoch 89/1000
2/2 [==============================] - 0s 29ms/step - loss: 13.4592 - mean_squared_error: 13.4592 - rmse: 3.6687 - mean_absolute_error: 3.1024 - val_loss: 10.1917 - val_mean_squared_error: 10.1917 - val_rmse: 3.1924 - val_mean_absolute_error: 2.6322 - lr: 1.2500e-04
Epoch 90/1000
2/2 [==============================] - 0s 33ms/step - loss: 15.1958 - mean_squared_error: 15.1958 - rmse: 3.8982 - mean_absolute_error: 3.1737 - val_loss: 10

In [11]:
# We want to try different training sample sizes: s
# (1453, 1000, 700, 350, 175)